In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

import pickle
import pandas as pd
import numpy as np

from pytorch import *

import matplotlib.pyplot as plt
from PIL import Image, ImageOps

In [2]:
with open("train.pkl", "rb") as f:
    df = pickle.load(f)
x = df[['x_pos', 'y_pos', 'x_vec', 'y_vec', 'x_vec2', 'y_vec2', 'x_dest', 'y_dest']].to_numpy()
y = df[['x_tar', 'y_tar']].to_numpy()

x = torch.Tensor(x)
y = torch.Tensor(y)
train = TensorDataset(x,y)
train_loader = DataLoader(train, batch_size=64, shuffle=False, drop_last=True)

In [3]:
with open("val.pkl", "rb") as f:
    df = pickle.load(f)
x = df[['x_pos', 'y_pos', 'x_vec', 'y_vec', 'x_vec2', 'y_vec2', 'x_dest', 'y_dest']].to_numpy()
y = df[['x_tar', 'y_tar']].to_numpy()

x = torch.Tensor(x)
y = torch.Tensor(y)
val = TensorDataset(x,y)
val_loader = DataLoader(train, batch_size=64, shuffle=False, drop_last=True)

In [4]:
# device = 'cuda' if torch.cuda.is_available() else 'cpu'
device = 'cpu'
print(f'Using {device} device')

Using cpu device


In [ ]:
input_dim = 8
output_dim = 2
hidden_dim = 100
layer_dim = 3
batch_size = 64
dropout = 0.2
n_epochs = 100
learning_rate = 1e-3
weight_decay = 1e-6

model_params = {'input_dim': input_dim,
                'hidden_dim' : hidden_dim,
                'layer_dim' : layer_dim,
                'output_dim' : output_dim,
                'dropout_prob' : dropout}

model = get_model('LSTM', model_params).to(device)

loss_fn = nn.MSELoss(reduction="mean")
optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)

opt = Optimization(model=model, loss_fn=loss_fn, optimizer=optimizer)
opt.train(train_loader, val_loader, batch_size=batch_size, n_epochs=n_epochs, n_features=input_dim)
opt.plot_losses()

# predictions, values = opt.evaluate(test_loader_one, batch_size=1, n_features=input_dim)

[1/100] Training loss: 302.8286	 Validation loss: 183.9059
[2/100] Training loss: 176.9178	 Validation loss: 156.9281
[3/100] Training loss: 158.7217	 Validation loss: 137.2163
[4/100] Training loss: 134.3241	 Validation loss: 109.0987
[5/100] Training loss: 108.1004	 Validation loss: 89.5474
[6/100] Training loss: 96.3270	 Validation loss: 82.2639
[7/100] Training loss: 90.8696	 Validation loss: 77.7359
[8/100] Training loss: 87.1689	 Validation loss: 77.0087
[9/100] Training loss: 86.2768	 Validation loss: 74.4224
[10/100] Training loss: 85.0656	 Validation loss: 75.4735


In [ ]:
# model = get_model('LSTM', model_params).to(device)
# model.load_state_dict(torch.load('models/2022-02-16_16-23-29'))

In [ ]:
def calc_next(x, y_pred):
    next = torch.tensor([[
        x[0] + y_pred[0],    # x_pos
        x[1] + y_pred[1],    # y_pos
        y_pred[0],           # x_vec
        y_pred[1],           # y_vec
        x[2],                # x_vec2
        x[3],                # y_vec2
        x[-2],               # x_dest
        x[-1]                # y_dest
    ]])
    return next

def simulate(model, inputs, iterations):
    for i in range(iterations):
        y_pred = model(inputs[i:])
        next = calc_next(inputs[i], y_pred[0])
        inputs = torch.cat((inputs, next))

In [ ]:
tis = torch.tensor([
        [[1050, 275, 0, 0, 0, 0, 1000, 470]],
        [[580*2, 50*2, 0, 0, 0, 0, 250*2, 280*2]],
        [[200*2, 225*2, 0, 0, 0, 0, 500*2, 250*2]],
        [[125*2, 200*2, 0, 0, 0, 0, 460*2, 50*2]],
        [[580*2, 50*2, 0, 0, 0, 0, 250*2, 300*2]],
        [[150*2, 240*2, 0, 0, 0, 0, 250*2, 300*2]]
        ]).type(torch.FloatTensor)

# tis[:,::2] /= 1280
# tis[:,1::2] /= 720

fig, ax = plt.subplots(figsize=(20,20))
im = Image.open("intersection2.png")
im = ImageOps.flip(im)
ax.set_xlim(0,1280)
ax.set_ylim(0,720)
ax.imshow(im, origin='lower')

inputs = tis[0].view([1, -1, 8])
for i in range(100):
    y_pred = model(inputs[i:])
    next = calc_next(inputs[i][0], y_pred[0]).view(1,-1,8)
    inputs = torch.cat((inputs, next))

ax.scatter(inputs[:,:,0], inputs[:,:,1])